In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="33abe262-85cf-48b9-81c3-83c7830e4a37",
    resource_group_name="eveML",
    workspace_name="EveML",
)

In [2]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure Machine Learning Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

You already have a cluster named cpu-cluster, we'll reuse it as is.


In [4]:
from azure.ai.ml.entities import Environment, BuildContext
import os

custom_env_name = "rl-particle-env"

env_docker_context = Environment(
    image="docker-contexts/python-and-pip",
    name=custom_env_name,
    description="Environment for the artifacts.",
)

ml_client.environments.create_or_update(env_docker_context)

env_docker_context.conda_file_path='xvfb-run -s "-screen 0 640x480x16 -ac +extension GLX +render" python'
env_docker_context.max_train_time = os.environ.get('AML_MAX_TRAIN_TIME_SECONDS', 2 * 60 * 60)

In [5]:
env_docker_context

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'rl-particle-env', 'description': 'Environment for the artifacts.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/rlcompute0/code/Users/eva.pardi', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f8e1cbd4610>, 'version': '4', 'latest_version': None, 'conda_file': None, 'image': 'docker-contexts/python-and-pip', 'build': None, 'inference_config': None, 'os_type': None, 'arm_type': 'environment_version', 'conda_file_path': 'xvfb-run -s "-screen 0 640x480x16 -ac +extension GLX +render" python', 'path': None, 'datastore': None, 'upload_hash': None, 'translated_conda_file': None, 'max_train_time': 7200})